In [35]:
import deepmatcher as dm
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/anhaidgroup/deepmatcher/master/examples/sample_data/itunes-amazon/train.csv
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/anhaidgroup/deepmatcher/master/examples/sample_data/itunes-amazon/validation.csv
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/anhaidgroup/deepmatcher/master/examples/sample_data/itunes-amazon/test.csv
!wget -qnc -P sample_data/itunes-amazon https://raw.githubusercontent.com/anhaidgroup/deepmatcher/master/examples/sample_data/itunes-amazon/unlabeled.csv

In [5]:
data_dir = 'sample_data/itunes-amazon'
datasets = dm.data.process(data_dir, train='train.csv', validation='validation.csv',
                           test='test.csv')
train = datasets[0]
validation = datasets[1]
test = datasets[2]


Reading and processing data from "sample_data/itunes-amazon/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "sample_data/itunes-amazon/validation.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "sample_data/itunes-amazon/test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [6]:
train_table = train.get_raw_table()
train_table.head()

,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,0,baby when the light ( david guetta & fred rist...,david guetta,pop life ( extended version ) [ bonus version ],"dance , music , rock , pop , house , electroni...",$ 1.29,‰ ãñ 2007 gum records,6:17,18-sep-07,revolver ( madonna vs. david guetta feat . lil...,david guetta,one love ( deluxe version ),dance & electronic,$ 1.29,( c ) 2014 swedish house mafia holdings ltd ( ...,3:18,"august 21 , 2009"
1,287,1,outversion,mark ronson,version,"pop , music , r & b / soul , soul , dance , ro...",$ 0.99,2007 mark ronson under exclusive license to so...,1:50,10-jul-07,outversion,mark ronson,version [ explicit ],pop,$ 0.99,( c ) 2011 j'adore records,1:50,"july 10 , 2007"
2,534,0,peer pressure ( feat . traci nelson ),snoop dogg,doggumentary,"hip-hop/rap , music , rock , gangsta rap , wes...",$ 1.29,"‰ ãñ 2011 capitol records , llc . all rights r...",4:07,29-mar-11,boom ( ( feat . t-pain ) [ edited ] ),snoop dogg,doggumentary [ edited ],"rap & hip-hop , west coast",$ 1.29,"( c ) 2011 capitol records , llc",3:50,"march 29 , 2011"
3,181,1,stars come out ( tim mason remix ),zedd,stars come out ( remixes ) - ep,"dance , music , electronic , house",$ 1.29,2012 dim mak inc .,5:49,20-may-14,stars come out ( dillon francis remix ),zedd,stars come out [ dillon francis remix ],dance & electronic,$ 1.29,2012 dim mak inc .,4:08,"may 20 , 2014"
4,485,0,jump ( feat . nelly furtado ),flo rida,r.o.o.t.s . ( deluxe version ),"hip-hop/rap , music",$ 1.29,‰ ãñ 2009 atlantic recording corporation for t...,3:28,30-mar-09,"yayo [ feat . brisco , billy blue , ball greez...",flo rida,r.o.o.t.s . ( route of overcoming the struggle...,rap & hip-hop,$ 1.29,"( c ) 2012 motown records , a division of umg ...",7:53,"march 30 , 2009"


## Define neural network models

In [7]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')

In [88]:
#run this cell if you want to load pre-trained model
hybrid_model.load_state('hybrid_model.pth')
hybrid_model.cuda()

MatchingModel(
  (attr_summarizers): ModuleMap(
    (Song_Name): Hybrid(
      (word_contextualizer): RNN(
        (rnn_groups): ModuleList(
          (0): GRU(300, 150, batch_first=True, bidirectional=True)
        )
        (dropouts): ModuleList(
          (0): Dropout(p=0)
        )
        (bypass_networks): ModuleList(
          (0): None
        )
        (input_dropout): NoMeta(
          (module): Dropout(p=0)
        )
      )
      (word_comparator): Attention(
        (alignment_networks): ModuleList(
          (0): AlignmentNetwork(
            (transform): Transform(
              (transforms): ModuleList(
                (0): Linear(in_features=300, out_features=300, bias=True)
                (1): Linear(in_features=300, out_features=300, bias=True)
              )
              (bypass_networks): ModuleList(
                (0): Bypass(
                  (highway_gate): Linear(in_features=300, out_features=300, bias=True)
                )
                (1): Bypass(


## Train models

In [9]:
hybrid_model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    5.9 | Load Time:    0.6 || F1:  39.02 | Prec:  28.40 | Rec:  62.34 || Ex/s:  49.29

===>  EVAL Epoch 1


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.6 | Load Time:    0.2 || F1:  67.92 | Prec:  62.07 | Rec:  75.00 || Ex/s: 132.04

* Best F1: 67.92452830188678
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 10 || Run Time:    5.1 | Load Time:    0.6 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  56.64

===>  EVAL Epoch 10


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 10 || Run Time:    0.6 | Load Time:    0.2 || F1:  85.71 | Prec:  84.00 | Rec:  87.50 || Ex/s: 129.94

---------------------

Loading best model...
Training done.


88.46153846153845

## Step 3.5 extract intermediate layers
In this step we want to evaluate the output of intermediate layers

In [10]:
# class to save intermediate layer output/input
class Hook():
    def __init__(self, module, backward=False):
        if backward==False:
            self.hook = module.register_forward_hook(self.hook_fn)
        else:
            self.hook = module.register_backward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.input = input
        self.output = output
    def close(self):
        self.hook.remove()

In [89]:
positives = dm.data.process(path='sample_data/itunes-amazon/',train='train.csv',
                            validation='altered_positives_samples.csv',test='positives_samples.csv',cache='pcache.pth')


Reading and processing data from "sample_data/itunes-amazon/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "sample_data/itunes-amazon/altered_positives_samples.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "sample_data/itunes-amazon/positives_samples.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [90]:
from deepmatcher.data import MatchingIterator
batch_size = 10
positives_splits = MatchingIterator.splits(positives,batch_size=batch_size)

In [91]:
positives_batches = [] 
for batch in positives_splits[2]:
    positives_batches.append(batch)

In [92]:
altered_positives = []
for batch in positives_splits[1]:
    altered_positives.append(batch)

In [93]:
hybrid_model.forward(positives_batches[0])

Variable containing:
-3.5324 -0.0297
-1.5120 -0.2491
-2.0955 -0.1313
-3.8832 -0.0208
-3.5521 -0.0291
-0.1272 -2.1246
-0.9190 -0.5090
-3.9984 -0.0185
-2.3539 -0.0998
-1.2216 -0.3492
[torch.cuda.FloatTensor of size 10x2 (GPU 0)]

In [94]:
hybrid_model.forward(altered_positives[0])

Variable containing:
-3.3979 -0.0340
-1.7124 -0.1990
-0.2488 -1.5130
-1.6829 -0.2056
-0.1177 -2.1977
-0.3870 -1.1367
-0.3977 -1.1143
-4.2716 -0.0141
-0.2067 -1.6781
-0.6373 -0.7523
[torch.cuda.FloatTensor of size 10x2 (GPU 0)]

In [95]:
attributes = ['Song_Name','Artist_Name','Album_Name','Price','CopyRight','Released','Genre','Time']

In [96]:
summarizers = []
comparators = []
for attr in attributes:
    summarizers.append(hybrid_model.attr_summarizers[attr])
    comparators.append(hybrid_model.attr_comparators[attr])

In [97]:
hookF = []
for summ in summarizers:
    hookF.append(Hook(summ))

In [98]:
def return_layer_input_output(hook_functions,batch,model):
    out= model(batch)
    layer_inputs = []
    layer_outputs = []
    for hook in hook_functions:
        layer_inputs.append(hook.input)
        layer_outputs.append(hook.output)
    return layer_inputs,layer_outputs

In [108]:
positives_batch_layer_inputs,positives_batch_layer_outputs = return_layer_input_output(hookF,
                                                                                     positives_batches[1],hybrid_model)

In [109]:
altered_batch_layer_inputs,altered_batch_layer_outputs = return_layer_input_output(hookF,
                                                                                  altered_positives[1],hybrid_model)

In [110]:
positives_summarizers_left_output = list(map(lambda x: x[0].data,positives_batch_layer_outputs))
positives_summarizers_right_output = list(map(lambda x: x[1].data,positives_batch_layer_outputs))

In [111]:
altered_summarizers_left_output = list(map(lambda x: x[0].data,altered_batch_layer_outputs))
altered_summarizers_right_output = list(map(lambda x:x[1].data,altered_batch_layer_outputs))

In [103]:
def calculate_distance_matrix(summarizers_left_output,summarizers_right_output):
    distance_mat = []
    for i in range(len(summarizers_left_output)):
        distances = []
        for j in range(batch_size):
            l_out = summarizers_left_output[i][j].data
            r_out = summarizers_right_output[i][j].data
            dist = distance.euclidean(l_out,r_out)
            distances.append(dist)
        distance_mat.append(distances)
    distance_mat = np.matrix(distance_mat)
    return distance_mat

In [112]:
distance_mat_positives = calculate_distance_matrix(positives_summarizers_left_output,positives_summarizers_right_output)
distance_mat_positives = distance_mat_positives.reshape((10,8))

In [113]:
distances_positives_df = pd.DataFrame(data = distance_mat_positives,columns=attributes)

In [114]:
distances_positives_df.head(10)

,Song_Name,Artist_Name,Album_Name,Price,CopyRight,Released,Genre,Time
0,0.000000,0.818525,0.353629,0.000000,0.000000,0.539159,0.000000,0.000000
1,0.000000,0.180276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.224907,1.275132,0.000026,0.741696
3,0.650884,0.099023,0.000040,0.000090,0.000023,0.000039,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.391206,0.005628,1.020680,0.720574,0.558888,0.005542,0.814779,0.808317
6,0.699374,0.628647,0.991943,0.512151,0.625245,0.524573,0.596789,0.657549
7,1.023974,0.655507,0.910311,0.588852,1.718542,1.337025,0.328442,1.655995
8,2.154017,0.246117,0.562215,2.061483,2.381361,0.872756,0.000000,0.000000
9,2.202635,0.000000,0.000000,2.343569,2.266645,3.207994,0.000000,0.000000


In [115]:
distance_mat_altered = calculate_distance_matrix(altered_summarizers_left_output,altered_summarizers_right_output)
distance_mat_altered = distance_mat_altered.reshape((10,8))

In [116]:
distances_altered_df = pd.DataFrame(data=distance_mat_altered,columns = attributes)

In [117]:
distances_altered_df.head(10)

,Song_Name,Artist_Name,Album_Name,Price,CopyRight,Released,Genre,Time
0,0.000000,0.818525,0.353629,0.000000,0.000000,0.539159,0.000000,0.000000
1,0.000000,0.180276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.224907,1.275132,0.000026,0.741696
3,0.650884,0.099023,0.000040,0.000090,0.000023,0.000039,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.391206,0.005628,1.020680,0.720574,0.558888,0.005542,0.814779,0.808317
6,0.699374,0.628647,0.991943,0.512151,0.625245,0.524573,0.596789,0.657549
7,1.023974,0.655507,0.910311,0.588852,1.718542,1.337025,0.328442,1.655995
8,2.154017,0.246117,0.562215,2.061483,2.381361,0.872756,2.629158,0.549303
9,1.141140,0.455519,2.756165,0.690089,0.510815,3.582113,3.026195,1.469071
